In [5]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Using Theano backend.


In [2]:
np.random.seed(7)

In [16]:
def read_data(path):
    df = pd.read_csv(path, parse_dates=['FECHA'], infer_datetime_format=True)
    df['CPRECIO'] = df[' CPRECIO '].map(lambda x: x.strip().replace(",", ""))
    df['CPRECIO'] = df['CPRECIO'].convert_objects(convert_numeric=True)
    df['COSTOPESOS'] = df[' COSTOPESOS ']
    df = df.drop([' CPRECIO ', ' COSTOPESOS '], axis=1)
    cols = df.columns.values 
    cols[-3] = "YEAR"
    df.columns = cols
    return df

def read_test_data(path):
    df = pd.read_csv(path, parse_dates=['FECHA'], infer_datetime_format=True)
    df['CPRECIO'] = df[' CPRECIO ']
    df['COSTOPESOS'] = df[' COSTOPESOS ']
    df = df.drop([' CPRECIO ', ' COSTOPESOS '], axis=1)
    cols = df.columns.values 
    cols[-3] = "YEAR"
    df.columns = cols
    return df

def calculate_extra_cols(df):
    df['total_price'] =  df['CPRECIO'] * df['#UNIDADES'] * df['CTIPOCAM01']
    return df

In [17]:
df = read_data('./BASEVENTAS2010A2015.csv')
df = calculate_extra_cols(df)

# Cleanup all the spaces
df["MARCA"] = df["MARCA"].map(lambda x: x.strip())
df["IDPRODUCTO"] = df["IDPRODUCTO"].map(lambda x: x.strip())

/Users/Grimi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [18]:
df = df[df["IDPRODUCTO"] == "25967"]
df = df.set_index("FECHA")
ts = df["#UNIDADES"]
ts = ts.resample("W").mean().sort_index().fillna(0)
ts = ts["2013":]

In [275]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(ts.values.reshape(-1, 1))
# dataset = scaler.fit_transform(np.arange(20).reshape(-1, 1))

In [276]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [281]:
# convert an array of values into a dataset matrix
# def create_dataset(dataset, look_back=1):
#     dataX, dataY = [], []
#     for i in range(len(dataset)-look_back-1):
#         a = dataset[i:(i + look_back), 0]
#         dataX.append(a)
#         dataY.append(dataset[i + look_back, 0])
#     return np.array(dataX), np.array(dataY)

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    start = len(dataset) - look_back - 1 - look_back
#     print start
    for i in range(start):
        a = dataset[i + 2:(i + look_back + 2), 0]
#         print "i", i, "i+lb", i + look_back 
        dataX.append(a)
        dataY.append(dataset[i + look_back + 2: (i + look_back*2) + 2, 0])
    return np.array(dataX), np.array(dataY)

# reshape into X=t and Y=t+1
look_back = 4
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [282]:
# create and fit the LSTM network
batch_size = 1
model = Sequential()
model.add(LSTM(5, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
model.add(LSTM(5, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dense(4))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, nb_epoch=200, batch_size=1, verbose=2)

Epoch 1/200
0s - loss: 0.0267
Epoch 2/200
0s - loss: 0.0250
Epoch 3/200
0s - loss: 0.0240
Epoch 4/200
0s - loss: 0.0243
Epoch 5/200
0s - loss: 0.0239
Epoch 6/200
0s - loss: 0.0243
Epoch 7/200
0s - loss: 0.0238
Epoch 8/200
0s - loss: 0.0236
Epoch 9/200
0s - loss: 0.0237
Epoch 10/200
0s - loss: 0.0233
Epoch 11/200
0s - loss: 0.0235
Epoch 12/200
0s - loss: 0.0235
Epoch 13/200
0s - loss: 0.0234
Epoch 14/200
0s - loss: 0.0235
Epoch 15/200
0s - loss: 0.0234
Epoch 16/200
0s - loss: 0.0234
Epoch 17/200
0s - loss: 0.0235
Epoch 18/200
0s - loss: 0.0234
Epoch 19/200
0s - loss: 0.0233
Epoch 20/200
0s - loss: 0.0233
Epoch 21/200
0s - loss: 0.0234
Epoch 22/200
0s - loss: 0.0234
Epoch 23/200
0s - loss: 0.0234
Epoch 24/200
0s - loss: 0.0233
Epoch 25/200
0s - loss: 0.0234
Epoch 26/200
0s - loss: 0.0232
Epoch 27/200
0s - loss: 0.0235
Epoch 28/200
0s - loss: 0.0233
Epoch 29/200
0s - loss: 0.0232
Epoch 30/200
0s - loss: 0.0232
Epoch 31/200
0s - loss: 0.0233
Epoch 32/200
0s - loss: 0.0233
Epoch 33/200
0s -

In [74]:
trainScore = model.evaluate(trainX, trainY, batch_size=batch_size, verbose=0)
model.reset_states()
print('Train Score: ', scaler.inverse_transform(np.array([[trainScore]])))

testScore = model.evaluate(testX, testY, batch_size=batch_size, verbose=0)
model.reset_states()
print('Test Score: ', scaler.inverse_transform(np.array([[testScore]])))

('Train Score: ', array([[ 0.00250468]]))
('Test Score: ', array([[ 1.68967603]]))


In [283]:
trainPredict = model.predict(trainX, batch_size=batch_size)
model.reset_states()
testPredict = model.predict(testX, batch_size=batch_size)
model.reset_states()

# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

ValueError: could not broadcast input array from shape (94,4) into shape (94,1)

In [ ]:
# plot baseline and predictions
plt.figure(figsize=(12,8))
plt.plot(dataset, alpha=0.5)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)

In [272]:
mydata = np.arange(22).reshape(-1,1)
# def create_dataset(dataset, look_back=1):
#     dataX, dataY = [], []
#     for i in range(len(dataset)-look_back-1):
#         a = dataset[i:(i + look_back), 0]
#         dataX.append(a)
#         dataY.append(dataset[i + look_back, 0])
#     return np.array(dataX), np.array(dataY)
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    start = len(dataset) - look_back - 1 - look_back
    print start
    for i in range(start):
        a = dataset[i + 2:(i + look_back + 2), 0]
        print "i", i, "i+lb", i + look_back 
        dataX.append(a)
        dataY.append(dataset[i + look_back + 2: (i + look_back*2) + 2, 0])
    return np.array(dataX), np.array(dataY)
trainX, trainY = create_dataset(mydata.reshape(-1,1), look_back=1)

19
i 0 i+lb 1
i 1 i+lb 2
i 2 i+lb 3
i 3 i+lb 4
i 4 i+lb 5
i 5 i+lb 6
i 6 i+lb 7
i 7 i+lb 8
i 8 i+lb 9
i 9 i+lb 10
i 10 i+lb 11
i 11 i+lb 12
i 12 i+lb 13
i 13 i+lb 14
i 14 i+lb 15
i 15 i+lb 16
i 16 i+lb 17
i 17 i+lb 18
i 18 i+lb 19


In [273]:
train_size = int(len(mydata) * 0.67)
test_size = len(mydata) - train_size
train, test = mydata[0:train_size,:], mydata[train_size:len(dataset),:]
print train

[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [13]]


In [274]:
trainX, trainY = create_dataset(train, 2)
testX, testY = create_dataset(test, 2)
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

9
i 0 i+lb 2
i 1 i+lb 3
i 2 i+lb 4
i 3 i+lb 5
i 4 i+lb 6
i 5 i+lb 7
i 6 i+lb 8
i 7 i+lb 9
i 8 i+lb 10
1
i 0 i+lb 2


In [269]:
print trainX
print trainY

[[[ 0]
  [ 1]]

 [[ 1]
  [ 2]]

 [[ 2]
  [ 3]]

 [[ 3]
  [ 4]]

 [[ 4]
  [ 5]]

 [[ 5]
  [ 6]]

 [[ 6]
  [ 7]]

 [[ 7]
  [ 8]]

 [[ 8]
  [ 9]]

 [[ 9]
  [10]]

 [[10]
  [11]]]
[ 2  3  4  5  6  7  8  9 10 11 12]
